In [1]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
encoded_y_train = to_categorical(y_train, num_classes=10)
y_test = y_test.reshape(10000,1)
shape = 'x_train shape: {} | encoded_y_train shape: {}\n'
shape += 'x_test shape : {} | y_test shape : {}'
print(shape.format(x_train.shape, encoded_y_train.shape, x_test.shape, y_test.shape))

def compile_and_train(model, num_epochs): 
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=encoded_y_train, batch_size=32, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    return history

x_train = x_train.reshape(60000,28,28,1)
input_shape = x_train[0,:,:,:].shape # 28 by 28
x_test = x_test.reshape(10000,28,28,1)

print(x_train.shape)
model_input = Input(shape=input_shape)

def second_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same', strides = 2)(model_input)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Dropout(0.25)(x)
    x = Conv2D(4*96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(4*96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(10, (1, 1), strides = 2)(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='second_cnn')
    
    return model

def first_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same')(model_input)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='first_cnn')
    
    return model

def evaluate_error(model):
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]  
  
    return error

/home/fineral/.conda/envs/myenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


x_train shape: (60000, 28, 28) | encoded_y_train shape: (60000, 10)
x_test shape : (10000, 28, 28) | y_test shape : (10000, 1)
(60000, 28, 28, 1)


In [2]:
first_model = first_cnn(model_input)
second_model = second_cnn(model_input)

first_model.load_weights('final_weights/first_cnn.05-0.23.hdf5')
second_model.load_weights('final_weights/second_cnn.06-0.25.hdf5')

models = [first_model,second_model]

In [3]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model
ensemble_model = ensemble(models, model_input)

In [4]:
evaluate_error(ensemble_model)


(10000, 10)
(10000, 1)


0.0906

In [5]:
evaluate_error(first_model)

(10000, 10)
(10000, 1)


0.1065

In [6]:
evaluate_error(second_model)

(10000, 10)
(10000, 1)


0.1062